# 大衍求一术：从一道古老的数学题学编程算法

精通编程的关键在于练习。而选择练习题也是有诀窍的。太小儿科的题目很快会让你感到没劲，题目太难就容易放弃。选择适当难度的题目会帮助你在巩固基础知识的同时也逐步培养算法设计的直觉和经验。所以我们经常会用一些有趣的数学问题来帮助我们学习编程。

以下这道题关于同余方程题出自中国古代的算经，可以说代表了当时数学的最高成就。不过我们不需要知道太多里面的数学史；这里我们只是会把它当成一个 Python 编程的练习来做。
我们会从中开发出一种高效的算法来解决这类问题。
这个过程中，我们会抽象和泛化问题、设计和优化算法、推导和运用一些数学定理等等。
虽然这些步骤听上去很吓人，但一步一步走下来你会发现其实并不太难。
而且计算机会帮我们把复杂的运算做掉，我们只需要想办法让我们的程序跑得更快就行。

希望你做完这题后会非常有成就感，因为你会发现你最终的算法在一些很难问题上比最初的版本快了不止上千倍！

那我们就开始啦。

我们今天的问题是这样的：

## 大衍求一术

《孙子算经》中有“物不知数”的问题：

    “今有物，不知其数，三三数之剩二，五五数之剩三，七七数之剩二，问物几何？”
    
### 翻译

    有一个数 x，除以 3 余 2 ，除以 5 余 3 ，除以 7 余 2 ，请问 x 是多少？

这类问题叫**同余方程**。它的解法被总结成**中国剩余定理**。不过我们不需要知道那么多，我们可以用编程的方法从头给出一个解法。

关于这个问题的渊源，我们可以参考百度百科：

https://baike.baidu.com/item/%E5%A4%A7%E8%A1%8D%E6%B1%82%E4%B8%80%E6%9C%AF/5523066

##  枚举法暴力破解

我们可以用枚举法来暴力破解这个问题

In [6]:
for x in range(200):
    if (x % 3 == 2) and (x % 5 == 3) and (x % 7 == 2):
        print("x =", x)
        break

x = 23


## 推广一下问题

我们要找到这样一个 `x` 使得
* 它除以 `d1` 的余数是 `m1`，即 `x % d1 == m1`  (原题中，除数 `d1 == 3`, 余数 `m1 == 2`)
* 它除以 `d2` 的余数是 `m2`，即 `x % d2 == m2`  (原题中，除数 `d2 == 5`, 余数 `m2 == 3`)
* 它除以 `d3` 的余数是 `m3`，即 `x % d3 == m3`  (原题中，除数 `d3 == 7`, 余数 `m3 == 2`)

推广后，我们可以把我们的算法写成一个函数。

In [10]:
def remainder_solver_brute_force(d1, m1, d2, m2, d3, m3):
    for x in range(200):
        if (x % d1 == m1) and (x % d2 == m2) and (x % d3 == m3):
            return x # 找到了

    return None # 无解

remainder_solver_brute_force(3, 2, 5, 3, 7, 2)

23

## 更难的问题

好，让我们试着把这个程序用在大一点除数和余数上。比如我们要解的问题是

* `d1 == 101`, `m1 == 2`
* `d2 == 1001`, `m2 == 3`
* `d3 == 10001`, `m3 == 5`

试试结果会怎样呢？

In [5]:
x = remainder_solver_brute_force(101, 2, 1001, 3, 10001, 5)
print(x)

None


居然无解。那问题出在哪里呢？

## 枚举的数量

原因是我们第二行那里 `for x in range(200)` 里的 200 数值太小了。其实换成 1000 或 10000 也是不够的。一般来说，我们要找到这三个除数 `d1, d2, d3` 的最小公倍数 `lcm(d1, d2, d3)` 才行。如果最小公倍数不好算，没关系，我们可以用三个除数的乘积来代替（三个除数两两互质的情形）。

在原题里，除数 3、5、7 的最小公倍数是 105，所以 200 没问题。而在后来那个难题里，101、1001、10001 的最小公倍数则是 $101 \times 1001 \times 10001 \approx 10^9$，自然 200 远远不够了。

不过好在呢，我们可以证明，如果**最多**只需要搜索最小公倍数个 $x$ 就够了。我们可以证明这个结论：

    如果 `x` 是一个解，`y = x + lcm(d1, d2, d3)` 也是解。

原因很简单，`d1` 可以整除 `lcm(d1, d2, d3)`，所以 `x`, `y` 自然对于 `d1` 是同余的。（`d2`, `d3` 也是一样）。

这个结论很有用，它是说我们不需要枚举超过 `lcm(d1, d2, d3)` 的数字 `x`，因为对这种数字，它和 `x' = x - lcm(d1, d2, d3)` 是一样的，而 `x'` 我们之前已经枚举过（假定我们是从小到大枚举 `x` 的话）。 

所以，有了这个结论，我们就可以放心大胆地写出一个通用的程序。

In [6]:
def remainder_solver_3var(d1, m1, d2, m2, d3, m3):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
        x % d3 == m3
    '''
    
    for x in range(d1*d2*d3):
        if (x % d1 == m1) and (x % d2 == m2) and (x % d3 == m3):
            return x
    return None

remainder_solver_3var(3, 2, 5, 3, 7, 2)

23

**大功告成！**

别急，对于大的除数，我们发现这个程序非常慢。

In [18]:
remainder_solver_3var(101, 2, 1001, 3, 10001, 5)

719681966

在我的机器上，它跑了大概20秒。

如果余数是随机的，可以估计我们平均下来我们要计算大概 `d1*d2*d3/2` 次才能算出正确结果，所以如果 `d1`、`d2`、`d3` 一大，程序就慢了。
下面我们着手来优化这个算法。

首先我们来考虑一些简单点的问题。以求得到一些启发。

## 单余数的问题

首先我们考虑一个除数的问题：

    找到 `x` 使得 `x` 除以 `d1` 的余数是 `m1`，即 `x % d1 == m1`.

结果很简单，就是 `x == m1`。

但是注意到，我们这里一下子就把答案说出来了，**没有**用刚才那个循环。如果把那个循环写出来，是这样子的：

In [4]:
def remainder_solver_1var(d1, m1):
    ''' return x such that
        x % d1 == m1
    '''
    
    for x in range(d1):
        if x % d1 == m1:
            return x
    return None

remainder_solver_1var(1, 2)

如果假定 m1 在0.. d1-1 之间均匀分布，这个循环平均来说要花大概 `d1/2` 此运算，比我们一下说出结果要费事很多。

这说明在一个余数的情况下，我们可以大幅优化机械循环的算法。

那么能否把这个思路推广到二余数和三余数的问题上呢？

## 二余数的问题

    找到 `x` 使得 `x` 除以 `d1` 的余数是 `m1`，除以 `d2` 的余数是 `m2`
    
作为一个初始算法，我们先把三余数的算法搬过来。把 `d3` 和 `m3` 的部分删去就是了。

In [3]:
def remainder_solver_2var(d1, m1, d2, m2):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
    '''
    
    for x in range(d1*d2):
        if (x % d1 == m1) and (x % d2 == m2):
            return x
    return -1

remainder_solver_2var(3, 2, 5, 3)

8

### 改进二余数问题的算法

这个算法在现在平均要走 `d1*d2/2` 次才能找到正确答案。

我们现在来想想怎么改进它。我们要同时满足两个方程

 * `x % d1 == m1`
 * `x % d2 == m2`

第一个方程的解就是单余数方程的解，这些解是有公式的：就是

   `x = m1 + d1*p`，其中 `p = 0, 1, 2, ...` 

如果我们从这些解中筛选第二个方程的解，就会比从全部整数中筛选要有效。

具体说，

 0. 我们可以从 `x = m1` 开始试， 因为它肯定满足第一个方程 `x % d1 == m1`，
我们只要测试它是否满足第二个方程 `x % d2 == m2` 。

 1. 试完之后，我们可以试 `x = m1 + d1`，因为它也满足第一个方程 `x % d1 == m1`，

 2. 再下来，我们可以试 `x = m1 + d1*2`，

 3. 再下来，`x = m1 + d1*3`，

 4. ……

这种试法有两个好处：

 * 需要试的数字少，每次跳 `d1` 个数字
 * 试的时候，循环内部只需要测试一个条件，就是第二个方程 `x % d2 == m2` 是否满足。第一个方程满足与否在构造这个算法时我们已经能保证了。

这种做法实际上是**先构造第一个方程的解的集合，然后从中挑一个第二个方程的解**。

让我们把这个改进版写下来：

In [8]:
def remainder_solver_2var_v2(d1, m1, d2, m2):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
    '''
    
    for x in range(m1, d1*d2, d1):
        if x % d2 == m2:
            return x
    return None

remainder_solver_2var_v2(3, 2, 5, 3)

8

注意第7行里，`range(m1, d1*d2, d1)` 从 `m1` 开始，每次递进 `d1`

所以这个循环最多跑 `d1*d2/d1 = d2` 次，平均一下的话是跑 `d2/2` 次，比起未优化的 `d1*d2/2` 次可好多了。

### 二余数问题的进一步改进

其实我们对这个算法还可以改进一下。
如果 `d1` 比 `d2` 小的话，我们可以把 `(d1, m1)` 和 `(d2, m2)` 互换一下，这样这个循环就只需要跑 `d1/2` 次了。

In [9]:
def remainder_solver_2var_v3(d1, m1, d2, m2):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
    '''
    
    if d1 < d2: # swap (d1, m1) and (d2, m2)
        d1, m1, d2, m2 = d2, m2, d1, m1
    
    for x in range(m1, d1*d2, d1):
        if x % d2 == m2:
            return x
    return None

remainder_solver_2var_v3(3, 2, 5, 3)

8

## 三余数问题

我们终于可以回来解决三余数问题了。

回顾一下解二余数问题时，我们是先把第一个方程的所有解用公式写出来，然后循环这些解，从中挑一个第二个方程的解。

那么我们现在也可以依样画葫芦，**先把所有前两个方程的解给列出来，然后从中筛选哪个是满足第三个方程的。**

好，现在让我们把具体步骤梳理一下。假定我们已经解出了前两个方程的解，也就是 `x % d1 == m1` 和 `x % d2 == m2` 的解，假定这个解是`x1_2`，那么**所有**前两个方程的解就是

 0. `x1_2`
 1. `x1_2 + d1*d2`
 2. `x1_2 + d1*d2*2`
 3. `x1_2 + d1*d2*3`
 4. `……`

这些解满足这个新的方程

 * `x % (d1*d2) == x1_2`
    
这样三余数问题也可以被归结为解以下两个方程：

 * `x % (d1*d2) == x1_2`
 * `x % d3 == m3`
    
而这个问题又是一个二余数问题。所以又可以套用我们对二余数问题的解！（给自己一点时间，想想是不是这么回事）

#### 小结

所以，三余数问题可以被阶段性地分为两个二余数问题

 * 求 `(d1, m1)` 和 `(d2, m2)` 的解 `(d1*d2, x1_2)`
 * 求 `(d1*d2, x1_2)` 和 `(d3, m3)` 的解
 
所以我们就有了以下这个算法

In [14]:
def remainder_solver_3var_v2(d1, m1, d2, m2, d3, m3):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
        x % d3 == m3
    '''
    
    x1_2 = remainder_solver_2var_v3(d1, m1, d2, m2)
    
    return remainder_solver_2var_v3(d1*d2, x1_2, d3, m3)

remainder_solver_3var_v2(3, 2, 5, 3, 7, 2)

23

### 复杂度

让我们来计算一下这个算法需要循环多少步。

* 对于第一个子问题：

    `(d1, m1)` 和 `(d2, m2)` 的解

它平均要循环 `min(d1, d2)/2` 步。

* 对于第二个子问题：

    `(d1*d2, x1_2)` 和 `(d3, m3)` 的解

它平均要走 `min(d1*d2, d3)/2` 步，假定 `d3` 比 `d1*d2` 小，那么就是 `d3/2` 步。

总共的平均步数就是 `min(d1, d2)/2 + d3/2`。

### 又一改进

这里又有一个优化的机会。

如果我们 `d3` 是三个除数里最大的那个，那么我们可以把它和 `d1` 或 `d2` 交换。
这样算法的平均循环步数就会被降到 `(d1 + d2)/2`。

这样我们就有一个新版本。

In [15]:
def remainder_solver_3var_v3(d1, m1, d2, m2, d3, m3):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
        x % d3 == m3
    '''
    dmax = max(d1, d2, d3)
    if dmax == d3: # swap d2 and d3
        d2, m2, d3, m3 = d3, m3, d2, m2

    x1_2 = remainder_solver_2var_v3(d1, m1, d2, m2)
    
    return remainder_solver_2var_v3(d1*d2, x1_2, d3, m3)

remainder_solver_3var_v3(3, 2, 5, 3, 7, 2)

23

## 速度测试

我们来测试一组比较大的 `d1, d2, d3` （不是一开头那个难问题）
 * `d1 == 11, m1 == 2`
 * `d2 == l01, m2 == 3`
 * `d3 == 10001, m3 == 5`

In [16]:
# 未优化
%timeit remainder_solver_3var(11, 2, 101, 3, 10001, 5)

963 ms ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
# 版本2
%timeit remainder_solver_3var_v2(11, 2, 101, 3, 10001, 5)

102 µs ± 4.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [18]:
# 版本3
%timeit remainder_solver_3var_v3(11, 2, 101, 3, 10001, 5)

11.6 µs ± 790 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


这里是以此运行的结果

版本   | 速度
------|------------------
未优化 | 963 ms ± 17.8 ms
版本2 v2 | 102 µs ± 4.74 µs
版本3 v3 | 11.6 µs ± 790 ns

我们不要只看数字，看一下单位，未优化的版本是 ms 量级的，版本2和3是 µs 量级的，本身差了三个量级！
所以版本3 `v3` 是最快的，而且在这个问题上要比未优化的问题快了近 $10^5$ 倍。

### 那个最难的问题

让我们回到一开头那个最难的问题：

In [11]:
# 未优化
remainder_solver_3var(101, 2, 1001, 3, 10001, 5)

719681966

In [20]:
# 版本2
remainder_solver_3var_v2(101, 2, 1001, 3, 10001, 5)

719681966

In [19]:
# 版本3
remainder_solver_3var_v2(101, 2, 10001, 5, 1001, 3)

719681966

最后两个版本瞬间出结果，但未优化版本用了20秒。

## 总结

我们这个例子里可以看到一些简单的优化可以让一个程序快几个数量级。所以我们可以理解为什么许多计算机学家对设计算法痴迷。

当然我们的例子里还有很多可以改进的地方，比如我们都是用乘积来代替最小公倍数等。

作为一个练习，可以思考一下如何用这个算法解决四余数问题和 $n$ 余数的问题。

研究计算机算法是一件非常烧脑又非常令人着迷的事，通过做题也会使你的编程和算法水平迅速提高。

如果你对类似的问题感兴趣，可以在以下网站找到更多的练习和学习资源：

 * 力扣 （编程题为主）
 https://leetcode-cn.com/problemset/all/
 
 做题时可以从难度较低和通过率较高的题目开始。
 如果做不出，也没关系，每道题的 “题解” 和 “评论” 页面里有很多有用的信息和别人的代码可以参考学习。

 
 * Project Euler （数学题为主）
 https://projecteuler.net/archives
 
 
 祝大家学习愉快！